## HTTP 프로토콜

* header/body 로 이루어짐
  - 로그인이 필요한 페이지의 경우, header에 특별한 정보를 정해진 포멧에 따라 넣어서 요청함
    - 쿠키 또는 세션


## 쿠키(Cookie)와 세션(Session)
 - HTTP Request 를 전송하면, 서버는 HTML 파일을 전달하고, 해당 요청은 완료된다.
 - HTTP 프로토콜은 연결을 유지하기 어려운 구조로 되어 있음<br>
   . 요청-응답 후 연결이 끊기는 구조(사용자 입력 정보등 상태 정보 활용이 어려운 구조임)
 - 상태 관리 정보를 저장하는 방식으로 쿠키(cookie)와 세션(session) 기법이 나옴

 <img src="https://www.fun-coding.org/00_Images/cookie_session.png" />

### 쿠키(cookie): 상태 정보를 클라이언트에 저장하는 방식
 1. HTML 페이지를 웹 서버에 요청
 2. 웹 서버에서 쿠키(cookie) 생성
 3. 웹 서버 응답(HTML 페이지를 돌려줄 때) HTTP 헤더에 쿠키를 포함해서 전송 
``` 
Set−Cookie: id=korea123
``` 
 4. 전달받은 쿠키는 웹브라우저에서 관리하고 있다가, 다음 요청 때 쿠키를 함께 전송
```
Cookie: id=korea123
```
 5. 서버에서는 쿠키 정보를 읽어 이전 상태 정보를 확인
 6. 필요시 서버가 쿠키 정보를 변경해서 응답시 변경된 쿠키와 함께 응답

### 세션(session): 상태 정보를 웹 서버에 저장하는 방식
1. 웹브라우저가 웹 서버에 요청하게 되면, 웹 서버가 해당 웹브라우저(클라이언트)에 유일한 ID(세션 ID)를 부여함
2. 해당 세션 ID는 응답(HTML 페이지를 돌려줄 때) HTTP 헤더에 넣어져 전달된다.
``` 
Set−Cookie: PHPSESSID=pi0fo9v2kdi5nuha3bcgiu8fq2
``` 
3. 웹브라우저는 이후 웹브라우저를 닫기 까지 해당 웹 서버 요청할 때 부여된 세션 ID를 HTTP 헤더에 넣어서 전달한다.
``` 
Cookie: PHPSESSID=pi0fo9v2kdi5nuha3bcgiu8fq2
``` 
4. 웹 서버는 세션 ID를 확인하고, 해당 세션에 관련된 정보를 확인한 후, HTML 페이지를 돌려준다.

### 로그인이 필요한 웹페이지 크롤링
1. 로그인이 필요한 웹페이지 요청시, 어떤 추가 정보가 함께 전송되는지 파악
   - 쿠키 또는 세션
2. 해당 추가 정보를 어떻게 크롤링시, 함께 요청을 할지를 코드레벨로 파악하기

## 한빛미디어 사이트 분석
1. 로그인할 때, ID/PW를 넣고, 요청
2. 서버에서 ID/PW가 맞는지 확인하고, 이에 맞는 세션값을 응답헤더에 넣어서 전송
3. 웹브라우저가 해당 응답헤더에서 세션값을 찾아서, 이후에 해당 사이트 요청시에는 세션값을 함께 전송
4. 서버에서 해당 세션값에 맞는 ID를 확인하고, 해당 ID에 맞는 웹페이지를 응답바디에 넣어서 전송
5. 웹브라우저가 해당 웹페이지를 보여줌

<div class="alert alert-block alert-success">
**<font color="blue" size="4em">실습 (세션에 대한 기본 이해)</font>**<br>
 1. [한빛미디어 회원가입](http://www.hanbit.co.kr/member/member_agree.html)<br>
 2. 크롬(Chrome) 브라우저로 www.hanbit.co.kr 페이지 오픈 후<br>
    - alt + command + i (맥북), F12 (윈도우) 누르고 Application -> Cookies -> http://www.hanbit.co.kr 이동<br>
    - PHPSESSID 확인<br>
 3. 한빛미디어 로그인 후<br>
    - PHPSESSID 값을 임의 값으로 수정<br>
 4. 마이한빛 메뉴 클릭 <br>
 <br>
 X PHPSESSID 값을 임의 값으로 수정했기 때문에, 로그인 정보가 필요한 마이한빛 페이지 오픈시, 서버측에서 해당 로그인 정보가 없으므로<br>
 로그인을 하라는 페이지로 이동함
</div>

### 유추하는데 도움이 되는 툴 
  - Chrome 개발자 도구 -> Network -> Check 'Preserver log' and Select 'Doc'
    1. Go www.hanbit.co.kr/index.html
    2. Add ID/PW and click 로그인
  - login.html -> login_proc.php -> index.html 로 호출됨을 확인할 수 있음
  - login_proc.php Request Method가 POST 이고, FORM 데이터에서 m_id, m_passwd 를 확인할 수 있음

In [4]:
import requests
from bs4 import BeautifulSoup

login_url = 'http://www.hanbit.co.kr/member/login_proc.php'
crawl_url = 'http://www.hanbit.co.kr/myhanbit/myhanbit.html'

# 세션 생성
session = requests.session()

params = dict()
params['m_id'] = ''            # 자신의 아이디
params['m_passwd'] = ''  # 자신의 패스워드

# 해당 세션에 POST 요청
res = session.post(login_url, data=params)

# 요청에 대한 응답 결과가 200 OK 가 나오지 않을 경우 다음 함수에서 바로 프로그램 종료됨
res.raise_for_status()

# 해당 세션으로 GET 요청 (이전 POST를 통해 받은 세션 쿠키값이 GET 요청시 추가되어 전송되므로 로그인된 상태의 요청과 동일하게 됨)
res = session.get(crawl_url)
# 응답 받은 HTML 파일 파싱
soup = BeautifulSoup(res.content, 'html.parser')

# 응답 받은 HTML 파일 중 마일리지 정보 테그를 CSS selector로 추출
data = soup.select('div.sm_mymileage > dl.mileage_section1 dd')

# select() 를 호출했으므로, data 는 리스트타입, 그래서 data를 for 문에 넣어서 각 아이템을 가져와서 출력하면 됨 (여기서는 결국 해당 리스트에는 하나의 아이템만 있을 것임)
for item in data:
    print (item.get_text())

In [7]:
print(data)

[]
